In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download quadeer15sh/celeba-face-recognition-triplets

Dataset URL: https://www.kaggle.com/datasets/quadeer15sh/celeba-face-recognition-triplets
License(s): CC0-1.0
 94% 218M/231M [00:01<00:00, 183MB/s]
100% 231M/231M [00:01<00:00, 139MB/s]


In [ ]:
!unzip celeba-face-recognition-triplets.zip -d /content/celeba

Streaming output truncated to the last 5000 lines.
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180490.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180492.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180495.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180501.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180515.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180517.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180518.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180531.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180532.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/180537.jpg  
  inflating: /content/celeba/CelebA FR Triplets/CelebA FR Triplets/images/18053

In [ ]:
base_dir="/content/celeba/CelebA FR Triplets/CelebA FR Triplets"
image_path=f"{base_dir}/images"
csv_path=f"{base_dir}/triplets.csv"

In [ ]:
!pip install torch torchvision torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 15.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os


In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.csv_file = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        anchor_path = os.path.join(self.img_dir, self.csv_file.iloc[idx, 0])
        pos_path = os.path.join(self.img_dir, self.csv_file.iloc[idx, 2])
        neg_path = os.path.join(self.img_dir, self.csv_file.iloc[idx, 4])

        anchor = Image.open(anchor_path).convert('RGB')
        pos = Image.open(pos_path).convert('RGB')
        neg = Image.open(neg_path).convert('RGB')

        if self.transform:
            anchor = self.transform(anchor)
            pos = self.transform(pos)
            neg = self.transform(neg)

        return anchor, pos, neg
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 128)

    def forward_once(self, x):
        return self.resnet(x)

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = torch.sqrt(torch.sum((output1 - output2) ** 2, dim=1))
        loss = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                          label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss
    def validate_model(model, dataloader, criterion):
        model.eval()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for anchor, pos, neg in dataloader:
                anchor, pos, neg = anchor.cuda(), pos.cuda(), neg.cuda()
                label_pos = torch.zeros(anchor.size(0)).cuda()
                label_neg = torch.ones(anchor.size(0)).cuda()

                output_anchor_pos, output_pos = model(anchor, pos)
                output_anchor_neg, output_neg = model(anchor, neg)

                loss_pos = criterion(output_anchor_pos, output_pos, label_pos)
                loss_neg = criterion(output_anchor_neg, output_neg, label_neg)
                loss = loss_pos + loss_neg
                total_loss += loss.item()

                dist_pos = torch.sqrt(torch.sum((output_anchor_pos - output_pos) ** 2, dim=1))
                dist_neg = torch.sqrt(torch.sum((output_anchor_neg - output_neg) ** 2, dim=1))

                correct_pos = (dist_pos < 0.5).sum().item()
                correct_neg = (dist_neg >= 0.5).sum().item()

                total_correct += correct_pos + correct_neg
                total_samples += 2 * anchor.size(0)

        avg_loss = total_loss / len(dataloader)
        accuracy = total_correct / total_samples
        return avg_loss, accuracy


In [ ]:
batch_size = 128
num_epochs = 10
learning_rate = 0.0001

save_dir = "/content"
os.makedirs(save_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
model = SiameseNetwork().cuda()

criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
dataset = SiameseDataset(csv_file=csv_path, img_dir=image_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


scaler = torch.cuda.amp.GradScaler()

best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for i, (anchor, pos, neg) in enumerate(dataloader):
        anchor, pos, neg = anchor.cuda(), pos.cuda(), neg.cuda()
        label_pos = torch.zeros(anchor.size(0)).cuda()
        label_neg = torch.ones(anchor.size(0)).cuda()

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            output_anchor_pos, output_pos = model(anchor, pos)
            output_anchor_neg, output_neg = model(anchor, neg)

            loss_pos = criterion(output_anchor_pos, output_pos, label_pos)
            loss_neg = criterion(output_anchor_neg, output_neg, label_neg)
            loss = loss_pos + loss_neg

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")

    val_loss, val_accuracy = validate_model(model, dataloader, criterion)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        save_path = os.path.join(save_dir, "best_accuracy_model.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'accuracy': best_accuracy
        }, save_path)
        print(f"Model saved with best accuracy: {best_accuracy:.4f} at {save_path}")

print("Training Complete.")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-17-e80ca64fc3d6>:113: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-17-e80ca64fc3d6>:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.c

Epoch [1/10], Step [1/128], Loss: 15.2059
Epoch [1/10], Step [11/128], Loss: 3.6208
Epoch [1/10], Step [21/128], Loss: 0.5985
Epoch [1/10], Step [31/128], Loss: 0.4874
Epoch [1/10], Step [41/128], Loss: 0.4636
Epoch [1/10], Step [51/128], Loss: 0.4447
Epoch [1/10], Step [61/128], Loss: 0.4424
Epoch [1/10], Step [71/128], Loss: 0.4150
Epoch [1/10], Step [81/128], Loss: 0.4236
Epoch [1/10], Step [91/128], Loss: 0.4070
Epoch [1/10], Step [101/128], Loss: 0.3761
Epoch [1/10], Step [111/128], Loss: 0.3649
Epoch [1/10], Step [121/128], Loss: 0.3222
Epoch [1/10], Average Loss: 1.2920
Validation Loss: 0.3304, Validation Accuracy: 0.7708
Model saved with best accuracy: 0.7708 at /content/best_accuracy_model.pth
Epoch [2/10], Step [1/128], Loss: 0.3316
Epoch [2/10], Step [11/128], Loss: 0.3124
Epoch [2/10], Step [21/128], Loss: 0.3288
Epoch [2/10], Step [31/128], Loss: 0.3119
Epoch [2/10], Step [41/128], Loss: 0.3232
Epoch [2/10], Step [51/128], Loss: 0.3297
Epoch [2/10], Step [61/128], Loss: 0.

In [ ]:
files.upload()

Saving test.csv to test (1).csv


{'test (1).csv': b'anchor,id1,pos,id2,neg,id3\n056279.jpg,1,108998.jpg,1,030848.jpg,496\n024091.jpg,1,000023.jpg,1,093653.jpg,9313\n122082.jpg,3,045833.jpg,3,188283.jpg,7200\n110393.jpg,3,021233.jpg,3,178433.jpg,4643\n101388.jpg,4,056784.jpg,4,105432.jpg,2988\n143743.jpg,4,107918.jpg,4,079773.jpg,5029\n093187.jpg,5,101049.jpg,5,141930.jpg,52\n018771.jpg,5,093951.jpg,5,151909.jpg,6247\n122677.jpg,6,003289.jpg,6,165740.jpg,4441\n147869.jpg,6,122930.jpg,6,103819.jpg,6147\n140464.jpg,7,091155.jpg,7,168229.jpg,620\n020284.jpg,7,003099.jpg,7,111670.jpg,2306\n108789.jpg,8,100773.jpg,8,073767.jpg,9544\n094456.jpg,8,137740.jpg,8,094731.jpg,2060\n038345.jpg,10,010990.jpg,10,073824.jpg,6118\n090423.jpg,10,150519.jpg,10,117513.jpg,1035\n028512.jpg,12,094568.jpg,12,189375.jpg,6814\n046666.jpg,12,028858.jpg,12,023205.jpg,1705\n102088.jpg,13,145289.jpg,13,191650.jpg,9739\n110430.jpg,13,128227.jpg,13,137603.jpg,1508\n122060.jpg,14,092944.jpg,14,059833.jpg,1200\n122873.jpg,14,069882.jpg,14,142590.jpg,6

In [ ]:
saved_model_path = '/content/celeba-final.pth'

model = SiameseNetwork().cuda()

checkpoint = torch.load(saved_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.eval()


<ipython-input-16-5b3b7bc4df86>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(saved_model_path)


SiameseNetwork(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [ ]:
test_csv_path = '/content/test.csv'
test_dataset = SiameseDataset(csv_file=test_csv_path, img_dir=image_path, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

def test_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for anchor, pos, neg in dataloader:
            anchor, pos, neg = anchor.cuda(), pos.cuda(), neg.cuda()
            label_pos = torch.zeros(anchor.size(0)).cuda()
            label_neg = torch.ones(anchor.size(0)).cuda()

            output_anchor_pos, output_pos = model(anchor, pos)
            output_anchor_neg, output_neg = model(anchor, neg)

            loss_pos = criterion(output_anchor_pos, output_pos, label_pos)
            loss_neg = criterion(output_anchor_neg, output_neg, label_neg)
            loss = loss_pos + loss_neg
            total_loss += loss.item()

            dist_pos = torch.sqrt(torch.sum((output_anchor_pos - output_pos) ** 2, dim=1))
            dist_neg = torch.sqrt(torch.sum((output_anchor_neg - output_neg) ** 2, dim=1))

            correct_pos = (dist_pos < 0.5).sum().item()
            correct_neg = (dist_neg >= 0.5).sum().item()

            total_correct += correct_pos + correct_neg
            total_samples += 2 * anchor.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples
    return avg_loss, accuracy



In [ ]:
test_loss, test_accuracy = test_model(model, test_dataloader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Test Loss: 0.0990, Test Accuracy: 0.9200
